# Testing Notebook

This notebook is a tool to test the accuracy of the model using the validation dataset that has been created.

In [ ]:
# Import thrid party libraries

import numpy as np
import os

# Import the core library from Precencia

from src.core.face_detector import FaceDetector
from src.core.face_encoder import FaceEncoder
from src.core.face_verificator import FaceVerificator

In [ ]:
face_detector = FaceDetector()
face_encoder = FaceEncoder()
face_recognizer = FaceVerificator()

In [ ]:
names = os.listdir(os.getcwd() + '\data\\testing')
print(names)
print("Length: {}".format(len(names)))

In [ ]:
# Create anchor embeddings

anchor_embeddings = []

for name in names:
  print(name)
  paths = os.listdir(os.getcwd() + '\data\\testing\\{}'.format(name))
  path = os.getcwd() + '\data\\testing\\{}\\'.format(name) + paths[0]
  extracted_face = face_detector.extract_face(image_path=path)
  anchor_embeddings.append(face_encoder.get_embedding(image_array=extracted_face))

np.save('./data/encodings/testing_anchor_embeddings_n40.npy', anchor_embeddings)

In [ ]:
name = "Ma_HuaTeng"

images = os.listdir(os.getcwd() + '\data\\testing\\{}'.format(name))
input_person_embedding = []
for idx, image in enumerate(images):
  print(idx)
  path = os.getcwd() + '\data\\testing\\{}\\{}'.format(name, image)
  extracted_face = face_detector.extract_face(image_path=path)
  input_person_embedding.append(face_encoder.get_embedding(image_array=extracted_face))

In [ ]:
# Create input embeddings

input_embeddings = []

for name in names:
  print(name)
  images = os.listdir(os.getcwd() + '\data\\testing\\{}'.format(name))
  images.pop(0) # Pop the anchor image
  input_person_embedding = []
  for image in images:
    path = os.getcwd() + '\data\\testing\\{}\\{}'.format(name, image)
    extracted_face = face_detector.extract_face(image_path=path)
    input_person_embedding.append(face_encoder.get_embedding(image_array=extracted_face))
  
  input_embeddings.append(input_person_embedding)

np.save('./data/encodings/testing_input_embeddings_n40.npy', input_embeddings)

In [ ]:
# Load testing anchor and input embeddings

anchor_embeddings = np.load('./data/encodings/testing_anchor_embeddings_n40.npy')
input_embeddings = np.load('./data/encodings/testing_input_embeddings_n40.npy')

print(np.shape(anchor_embeddings))
print(np.shape(input_embeddings))

In [ ]:
def predict_user(face_embedding):
  similarities = [face_recognizer.compare(
        face_embedding, anchor_embedding) for anchor_embedding in anchor_embeddings]
  
  max_similarity = max(similarities)
  print("Maximum similarities: {}".format(max_similarity))
  max_similarity_index = similarities.index(max_similarity)

  if max_similarity > 0.97:
    user = names[max_similarity_index]
    return user
    
  return None

In [ ]:
true_predicted = []
false_predicted = []
not_predicted = []

for idx, input_person_embedding in enumerate(input_embeddings):
  current_user = names[idx]
  true_predicted_user = 0
  false_predicted_user = 0
  not_predicted_user = 0
  for face_embedding in input_person_embedding:
    user = predict_user(face_embedding)
    if user is not None:
      print("Actual User: {}, Predicted User: {}".format(current_user, user))
      if user == current_user:
        true_predicted_user += 1
      else:
        false_predicted_user += 1
    else:
      not_predicted_user += 1
  
  true_predicted.append(true_predicted_user)
  false_predicted.append(false_predicted_user)
  not_predicted.append(not_predicted_user)
  print("\n")

In [ ]:
print(true_predicted)
print(false_predicted)
print(not_predicted)

true_predicted_percentage = np.sum(true_predicted) / (len(names) * 5)
false_predicted_accuracy = np.sum(false_predicted) / (len(names) * 5)

print("True Predicted: {}".format(true_predicted_percentage))
print("False Predicted: {}".format(false_predicted_accuracy))